In [1]:
# Importing Dependencies
import pandas as pd
import gmaps
import requests
from config import g_key

In [2]:
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Kaitangata,NZ,2021-09-16 00:18:36,-46.2817,169.8464,44.35,88,78,4.34
1,1,Port Alfred,ZA,2021-09-16 00:22:01,-33.5906,26.8910,55.89,82,39,8.32
2,2,Cape Town,ZA,2021-09-16 00:20:58,-33.9258,18.4232,54.88,88,20,3.00
3,3,Nanortalik,GL,2021-09-16 00:21:18,60.1432,-45.2371,39.06,76,1,5.21
4,4,Tigil,RU,2021-09-16 00:22:03,57.8000,158.6667,53.89,49,20,6.58


In [3]:
# Configure gmaps to use API key
gmaps.configure(api_key=g_key)

In [4]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp], dissipating=False,
                                max_intensity=300, point_radius=4)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [5]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [6]:
# Heatmap of percent cloudiness
locations = city_data_df[["Lat", "Lng"]]
cloudiness = city_data_df["Cloudiness"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=cloudiness, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
# Heatmap of windiness
locations = city_data_df[["Lat", "Lng"]]
wind = city_data_df["Wind Speed"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=wind, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
# Ask customer temperature range
#min_temp = float(input("What is the lowest temperature you would like for your trip? "))
#max_temp = float(input("What is the highest temperature you would like for your trip? "))
min_temp = 75
max_temp = 90

In [9]:
preferred_cities_df = city_data_df.loc[(city_data_df['Max Temp'] >= min_temp) & 
                                    (city_data_df['Max Temp'] <= max_temp)]

preferred_cities_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
6,6,Samarai,PG,2021-09-16 00:22:04,-10.6167,150.6667,79.25,72,84,16.37
7,7,Pilar,BR,2021-09-16 00:22:04,-9.5972,-35.9567,76.93,85,46,3.65
8,8,Westport,US,2021-09-16 00:20:18,41.1415,-73.3579,78.62,87,1,1.01
12,12,Hithadhoo,MV,2021-09-16 00:22:07,-0.6000,73.0833,82.00,75,68,11.34
14,14,Arraial Do Cabo,BR,2021-09-16 00:17:18,-22.9661,-42.0278,75.16,88,75,10.36


In [10]:
preferred_cities_df.count()

City_ID       187
City          187
Country       187
Date          187
Lat           187
Lng           187
Max Temp      187
Humidity      187
Cloudiness    187
Wind Speed    187
dtype: int64

In [15]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
6,Samarai,PG,79.25,-10.6167,150.6667,
7,Pilar,BR,76.93,-9.5972,-35.9567,
8,Westport,US,78.62,41.1415,-73.3579,
12,Hithadhoo,MV,82.00,-0.6000,73.0833,
14,Arraial Do Cabo,BR,75.16,-22.9661,-42.0278,
24,Saint Anthony,US,75.13,45.0205,-93.2180,
28,Vaini,TO,80.76,-21.2000,-175.2000,
29,Jacareacanga,BR,77.54,-6.2222,-57.7528,
48,Kapaa,US,87.78,22.0752,-159.3190,
51,West Bay,KY,84.27,19.3667,-81.4167,


In [16]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [17]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

        
hotel_df.head

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


<bound method NDFrame.head of                 City Country  Max Temp      Lat       Lng  \
6            Samarai      PG     79.25 -10.6167  150.6667   
7              Pilar      BR     76.93  -9.5972  -35.9567   
8           Westport      US     78.62  41.1415  -73.3579   
12         Hithadhoo      MV     82.00  -0.6000   73.0833   
14   Arraial Do Cabo      BR     75.16 -22.9661  -42.0278   
..               ...     ...       ...      ...       ...   
548       Mahibadhoo      MV     82.74   3.7833   72.9667   
551     Marsa Matruh      EG     76.71  31.3525   27.2453   
553           Pangai      TO     82.51 -19.8000 -174.3500   
554    Belle Fourche      US     86.34  44.6714 -103.8522   
555    Alice Springs      AU     85.55 -23.7000  133.8833   

                          Hotel Name  
6                          Nuli Sapi  
7    Pousada e churrascaria Manguaba  
8                   The Westport Inn  
12                 Scoop Guest House  
14               Pousada Porto Praia  
.. 

In [18]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [19]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""


In [20]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [21]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))